## João Pedro Rodrigues Freitas - 11316552

### TODO
- Fazer os mini batches (train, test)
- Fazer o one-hot
- conferir normalização - OK
- Não passar func ativ na ultima camada - OK
- Fazer o espelhamento automático - OK

In [1262]:
import numpy as np
import matplotlib.pyplot as plt

## Data

In [1263]:
class Data():
    def __init__(self, X: np.ndarray) -> None:
        self.data = X

    def getData(self) -> np.ndarray:
        return self.data

    def setTrainRatio(self, ratio: float) -> None:
        self.train_ratio = ratio

    def getTrainRatio(self) -> float:
        return self.train_ratio
    
    def getAttrSize(self) -> int:
        return self.data.shape[1]
    
    def shuffleData(self) -> None:
        self.data = np.random.permutation(self.data)

    def getTrainData(self) -> np.ndarray:
        return self.data[:int(self.data.shape[0] * self.train_ratio)]
    
    def getTestData(self) -> np.ndarray:
        return self.data[int(self.data.shape[0] * self.train_ratio):]

    def normalize(self) -> None:
        '''Normaliza cada atributo para o intervalo [0, 1]'''
        for i in range(self.getAttrSize()):
            self.data[:,i] = (self.data[:,i] - np.min(self.data[:,i])) / (np.max(self.data[:,i]) - np.min(self.data[:,i]))

## Layer

In [1264]:
class Layer():
    def __init__(self, n_neurons: int, actv_func, inputs = None, lastLayer = False) -> None:
        self.n_neurons = n_neurons
        self.W = None
        self.b = None

        self.inputs = inputs
        self.shape = None
        self.inputShape = None

        self.actv_func = actv_func

        self.out = None
        self.actv = None

        self.lastLayer = lastLayer

        self._setWeights()

    def _setWeights(self) -> None:  
        if self.W is None and self.inputs is not None:
            self.inputShape = self.inputs.shape

            self.shape = (self.inputShape[-1], self.n_neurons)
            # Inicializa os pesos e bias com distribuição uniforme
            # entre -0.5 e 0.5
            self.W = np.random.rand(self.shape[0], self.shape[1]) - 0.5
            self.b = np.random.rand(self.n_neurons) - 0.5

    def process(self, inputs):
        '''Forward da camada'''
        self.inputs = inputs
        self._setWeights()
        self.out = np.dot(self.inputs, self.W) + self.b
        if not self.lastLayer:
            self.actv = self.actv_func(self.out)
        else:
            self.actv = self.out

    def getOutput(self):
        return self.out
    
    def getActivation(self):
        return self.actv
        


# Autoencoder

In [1265]:
class Autoencoder():
    def __init__(self) -> None:
        self.layers = np.array([], dtype=Layer)

        self.epochs = None
        self.lr = None

        self.inputs = None # X
        self.targets = None # X

        self.outputs = []
        self.ativations = [] # X_Hat

    def addLayer(self, layer: Layer) -> None:
        self.layers = np.append(self.layers, layer)

    # TODO: separar entre encoder e decoder
    def forward(self, inputs):
        self.ativations = []
        self.outputs = []

        self.ativations.append(inputs)
        self.outputs.append(inputs)

        for layer in self.layers:
            layer.process(inputs)
            inputs = layer.getActivation()
            output = layer.getOutput()

            self.outputs.append(output)
            self.ativations.append(inputs)

            # inputs = layer.getActivation()

        return inputs
    
    # TODO: separar entre encoder e decoder
    def backward(self) -> None:
        i = self.layers.size
        n = self.inputs.shape[0]

        err = 2 * (self.targets - self.ativations[-1]) # x - x_hat

        # TODO: dividir por n_samples

        for layer, z in zip(self.layers[::-1], self.outputs[::-1]):
            delta = err

            if (i != self.layers.size):
                delta *= layer.actv_func(z, derivative=True)

            # delta = err * layer.actv_func(x_hat, derivative=True)

            err = np.dot(delta, layer.W.T) # Atualiza o erro pro layer anterior

            if i > 0:
                prev_ativ = self.ativations[i-1]
                # dW = np.dot(prev_output.T, delta) / n
                # db = np.sum(delta, axis=0) / n
                dW = np.dot(prev_ativ.T, delta) / n
                db = delta.mean()

                layer.W += self.lr * dW
                layer.b += self.lr * db


            i -= 1
        
    def fit(self, inputs, targets, lr: float = 0.01, epochs: int = 100) -> None:
        self.inputs = inputs
        self.targets = targets
        self.lr = lr

        for epoch in range(epochs):
            self.forward(inputs)
            self.backward()

            if epoch % 10 == 0:
                print(f'Epoch: {epoch}')
                print(f'Loss: {np.mean(np.square(self.targets - self.ativations[-1]))}')


    def predict(self, inputs) -> np.ndarray:
        self.forward(inputs)
        return self.ativations[-1]

In [1266]:
def sigmoid(x, derivative=False):
    if derivative:
        return sigmoid(x) * (1 - sigmoid(x))

    positives = x >= 0
    negatives = ~positives
    
    exp_x_neg = np.exp(x[negatives])
    
    y = x.copy()
    y[positives] = 1 / (1 + np.exp(-x[positives]))
    y[negatives] = exp_x_neg / (1 + exp_x_neg)
    
    return y

def createModel(inputDim, layers, actv_func):
    model = Autoencoder()

    # Adiciona as camadas do encoder
    for i in range(len(layers)):
        model.addLayer(Layer(layers[i], actv_func))

    # Adiciona as camadas do decoder
    for i in range(len(layers)-2, -1, -1):
        model.addLayer(Layer(layers[i], actv_func))

    # Adiciona a camada de saída
    model.addLayer(Layer(inputDim, actv_func, lastLayer = True))

    return model

## Main

In [1267]:
X = np.vstack([np.random.normal(1, 0.5, [100,64]),
                np.random.normal(-2, 1, [100,64]),
                np.random.normal(3, 0.75, [100,64])])

data = Data(X)

data.normalize()
data.shuffleData()
data.setTrainRatio(0.8)

inputDim = data.getAttrSize()

# Número de neurônios em cada camada escondida
# A última camada dessa lista é a dimensão latente
# O espelhamento é feito automaticamente
# Exemplo:
# Para um X com 64 atributos, se hiddenLayers = [32, 16], a rede terá a seguinte estrutura:
# [64, 32, 16, 32, 64]
# Em que o primeiro valor é a dimensão de entrada e o último é a dimensão de saída
hiddenLayers = [32, 16]

autoencoder = createModel(inputDim, hiddenLayers, actv_func=sigmoid)

# autoencoder = createModel([inputDim] + teste + [inputDim], actv_func=sigmoid)
autoencoder.fit(data.getTrainData(), data.getTrainData(), lr=0.01, epochs=1000)

Epoch: 0
Loss: 1.174781053735059
Epoch: 10
Loss: 0.07346832254839122
Epoch: 20
Loss: 0.054883106829943795
Epoch: 30
Loss: 0.054287715813658356
Epoch: 40
Loss: 0.054109685520874605
Epoch: 50
Loss: 0.05393917623647268
Epoch: 60
Loss: 0.05376554353960545
Epoch: 70
Loss: 0.053588085940637985
Epoch: 80
Loss: 0.05340638222233869
Epoch: 90
Loss: 0.05322004302429386
Epoch: 100
Loss: 0.05302870316946081
Epoch: 110
Loss: 0.05283201995933295


Epoch: 120
Loss: 0.05262967127279914
Epoch: 130
Loss: 0.05242135322555697
Epoch: 140
Loss: 0.05220677731291112
Epoch: 150
Loss: 0.05198566698420885
Epoch: 160
Loss: 0.05175775362855188
Epoch: 170
Loss: 0.05152277199142546
Epoch: 180
Loss: 0.05128045508772273
Epoch: 190
Loss: 0.051030528724017234
Epoch: 200
Loss: 0.05077270578673397
Epoch: 210
Loss: 0.050506680487994976
Epoch: 220
Loss: 0.050232122783203416
Epoch: 230
Loss: 0.049948673181400734
Epoch: 240
Loss: 0.0496559381607544
Epoch: 250
Loss: 0.04935348637905796
Epoch: 260
Loss: 0.04904084583650721
Epoch: 270
Loss: 0.048717502110027455
Epoch: 280
Loss: 0.04838289774012739
Epoch: 290
Loss: 0.04803643281719464
Epoch: 300
Loss: 0.04767746678776938
Epoch: 310
Loss: 0.04730532148462741
Epoch: 320
Loss: 0.046919285377941086
Epoch: 330
Loss: 0.04651861904745416
Epoch: 340
Loss: 0.04610256188545928
Epoch: 350
Loss: 0.04567034005450941
Epoch: 360
Loss: 0.04522117573871775
Epoch: 370
Loss: 0.04475429773923663
Epoch: 380
Loss: 0.04426895346873

# Testar o modelo

In [1268]:
X = data.getTestData()
X_hat = autoencoder.predict(X)

err = np.mean(np.square(X - X_hat))

print(f'Erro quadrático médio: {err * 100:.3f}%')

Erro quadrático médio: 0.883%
